# Wikipedia Page Views Pipeline
 Name and ID: Balint Decsi, 2506626

## Choose Username

In [ ]:
USERNAME = "balintd-de1"

## Setup and Imports

In [2]:
import datetime
import json

import boto3
import requests

## Extract: Retrieve Data from Wikipedia API

**API Documentation:** https://doc.wikimedia.org/generated-data-platform/aqs/analytics-api/reference/edits.html

In [3]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-11"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/11
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"11","articles":[{"article":"Main_Page","views":6005182,"rank":1},{"article":"Dharmendra","views":1392953,"rank":2},{"article":"Special:Search","views":953512,"rank":3},{"article":".xxx","views":615108,"rank":4},{"article":"Google_Chrome","views":332922,"rank":5},{"article":"James_A._Garfield","views":266550,"rank":6},{"article":"Wikipedia:Featured_pictures","views":256622,"rank":7},{"article":"SS_Edmund_F...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 56358


## Transform: Process Raw Data into JSON Lines

Convert the raw API response into a structured JSON Lines format suitable for analytics. Each line is a valid JSON object representing one page's edit statistics.

In [4]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:5]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6005182, "rank": 1, "date": "2025-11-11", "retrieved_at": "2025-12-10T19:06:47.004304"}
{"title": "Dharmendra", "views": 1392953, "rank": 2, "date": "2025-11-11", "retrieved_at": "2025-12-10T19:06:47.004304"}
{"title": "Special:Search", "views": 953512, "rank": 3, "date": "2025-11-11", "retrieved_at": "2025-12-10T19:06:47.004304"}
{"title": ".xxx", "views": 615108, "rank": 4, "date": "2025-11-11", "retrieved_at": "2025-12-10T19:06:47.004304"}
{"title": "Google_Chr...


---
## 1: Create an S3 Bucket

**Documentation:** [create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/create_bucket.html)

In [5]:
S3_WIKI_BUCKET = USERNAME + "-wikidata"
s3 = boto3.client("s3")

# List existing buckets
# Note: If this raises NoCredentialsError, you need to configure AWS credentials
response = s3.list_buckets()
bucket_names = [bucket["Name"] for bucket in response["Buckets"]]
if S3_WIKI_BUCKET not in bucket_names:
    s3.create_bucket(
        Bucket=S3_WIKI_BUCKET,
        CreateBucketConfiguration={"LocationConstraint": boto3.session.Session().region_name},
    )
    print(f"Created new bucket: {S3_WIKI_BUCKET}")
else:
    print(f"Using existing bucket: {S3_WIKI_BUCKET}")

# Test
assert USERNAME != "<username>", "Please set your USERNAME at the top of the notebook"
assert S3_WIKI_BUCKET.endswith("-wikidata"), "Bucket name must end with '-wikidata'"

try:
    s3.head_bucket(Bucket=S3_WIKI_BUCKET)
    print(f"Bucket {S3_WIKI_BUCKET} exists!")
except Exception as e:
    print(f"Bucket {S3_WIKI_BUCKET} not found: {e}")
    raise


ClientError: An error occurred (InvalidBucketName) when calling the CreateBucket operation: The specified bucket is not valid.

---
## 2: Upload JSON Lines to S3

**Documentation:** [put_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/put_object.html)

In [ ]:
# Upload json_lines directly to S3
file_path = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=file_path,
    Body=json_lines,
)

# Test
expected_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"
try:
    s3.head_object(Bucket=S3_WIKI_BUCKET, Key=expected_key)
    print(f"File uploaded successfully to s3://{S3_WIKI_BUCKET}/{expected_key}")
except Exception as e:
    print(f"File not found at s3://{S3_WIKI_BUCKET}/{expected_key}")
    raise

File uploaded successfully to s3://balintd-de1-wikidata/raw-views/raw-views-2025-11-11.json
